In [1]:
#Implementation Of Radial Basis Function For Images

import torch
import torch.nn as nn
import torchvision

import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
transform = transforms.Compose( [transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [3]:
train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

AttributeError: module 'torch' has no attribute '_six'

In [4]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)

In [5]:
class RBF(nn.Module):

    def __init__(self,input_dim,output_dim,hidden_dim):
        super().__init__()
        self.first_layer = nn.Linear(input_dim,hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim,output_dim)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.s = 0.02
        self.centers = nn.Parameter(torch.randn(input_dim,hidden_dim))

    def formula(self,x,y):
        return torch.exp(-self.s * (x.unsqueeze(1) - y).pow(2).sum(dim=1))

    def forward(self,x):
        x = x.view(-1, 3072)
        out1 = self.first_layer(x)
        out3 = self.hidden_layer(out1)
        a = self.formula(out3, self.centers)
        return a

In [6]:
model = RBF(input_dim=3*32*32,output_dim=10,hidden_dim=10)
print(model)

RBF(
  (first_layer): Linear(in_features=3072, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=10, bias=True)
)


In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1): 
    for data in trainset:  
        X, y = data  
        model.zero_grad()  
        output = model(X.view(-1,3*32*32)) 
        loss = loss_function(output, y)  
        loss.backward()  
        optimizer.step() 
        print(loss)  

tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.30

In [8]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [14]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = 'data'

In [15]:
callbacks = [TensorBoard(log_dir=log_dir,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]